# 1. Instalação de Dependências

In [ ]:
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pyarrow.parquet as pq

# 2. Exploração inicial

In [ ]:
df = pd.read_excel('./consumo_materiais2019.xlsx')

df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
for coluna in df.columns:
    print(f"\n{coluna}: {df[coluna].nunique()} valores únicos")

# 3. Exploração avançada

## 3.1 Visualisar itens mais frequentes

In [ ]:
produtos_mais_frequentes = df['descricao'].value_counts()
print(produtos_mais_frequentes)

In [ ]:
top_10_produtos = produtos_mais_frequentes.head(10)

top_10_produtos.plot(kind='barh', figsize=(10, 5))
plt.title('10 Produtos mais frequentes')
plt.xlabel('Quantidade')
plt.ylabel('Produto')
plt.show()

## 3.2 Quantidade total de cada produto

In [ ]:
quantidade_total = df['qtd-saida'].sum()
print(f'Quantidade total de produtos comprados: {quantidade_total}')

In [ ]:
df.groupby('data-mov')['qtd-saida'].sum().plot(kind='line', figsize=(10, 5))
plt.title('Quantidade de produtos por data')
plt.xlabel('Data')
plt.ylabel('Quantidade')
plt.show()

## 3.3 Requisições mais frequentes

In [ ]:
historicos_mais_frequentes = df['historico'].value_counts()
print(historicos_mais_frequentes)

In [ ]:
historicos_mais_frequentes.plot(kind='pie', autopct='%1.1f%%', figsize=(8, 8))
plt.title('Tipos de requisição mais frequentes')
plt.show()

## 3.4 Correlação entre variáveis

In [ ]:
df_numeric = df[['cod-reduzido-produto', 'documento', 'qtd-saida']]
correlation_matrix = df_numeric.corr()

plt.figure(figsize=(8, 6))

sns.heatmap(correlation_matrix, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("Mapa de Calor da Correlação entre Variáveis Numéricas")
plt.show()

## 3.5 Relação de consumo de materiais por período

In [ ]:
df['data-mov'] = pd.to_datetime(df['data-mov'], errors='coerce', dayfirst=True)
df = df.dropna(subset=['data-mov'])

df['ano_mes'] = df['data-mov'].dt.to_period('M')

consumo_por_periodo = df.groupby('ano_mes')['qtd-saida'].sum().reset_index()

consumo_por_periodo['ano_mes'] = consumo_por_periodo['ano_mes'].dt.to_timestamp()

plt.figure(figsize=(12, 6))
plt.plot(consumo_por_periodo['ano_mes'], consumo_por_periodo['qtd-saida'], marker='o', color='b')
plt.xlabel('Período (Ano e Mês)')
plt.ylabel('Quantidade Total de Saída')
plt.title('Consumo de Materiais por Período')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

## 3.6 Relação entre tipo de pedido e data

In [ ]:
df['data-mov'] = pd.to_datetime(df['data-mov'], errors='coerce', dayfirst=True)

df = df.dropna(subset=['data-mov'])

requisicoes_por_data = df.groupby(['data-mov', 'historico']).size().unstack(fill_value=0)

plt.figure(figsize=(12, 6))
requisicoes_por_data.plot(kind='line', marker='o', figsize=(14, 7))
plt.xlabel('Data')
plt.ylabel('Quantidade de Requisições')
plt.title('Relação entre Tipo de Requisição e Data')
plt.xticks(rotation=45)
plt.legend(title='Tipo de Requisição')
plt.tight_layout()
plt.show()

# 4. Manipulação dos dados

## 4.1 Conversão de variáveis categóricas em numéricas

In [ ]:
df_encoded = df.copy()

label_encoder = LabelEncoder()
for column in ['cod-estruturado-produto', 'descricao', 'un-medida', 'historico']:
    df_encoded[column] = label_encoder.fit_transform(df_encoded[column])

print(df_encoded.head())

## 4.2 Remoção de outliers

In [ ]:
df_no_outliers = df.copy()

numerical_cols = ['cod-reduzido-produto', 'documento', 'qtd-saida']

for col in numerical_cols:
    Q1 = df_no_outliers[col].quantile(0.25)
    Q3 = df_no_outliers[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_no_outliers = df_no_outliers[(df_no_outliers[col] >= lower_bound) & (df_no_outliers[col] <= upper_bound)]

print(df_no_outliers.head())
print(f"Número de linhas antes da remoção de outliers: {df.shape[0]}")
print(f"Número de linhas após a remoção de outliers: {df_no_outliers.shape[0]}")

## 4.3 Padronização e PCA

In [ ]:
numerical_cols = ['cod-reduzido-produto', 'documento', 'qtd-saida']
df_numeric = df_no_outliers[numerical_cols]

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_numeric)

pca = PCA(n_components=2)  # Reduzir para 2 componentes principais, por exemplo
principal_components = pca.fit_transform(df_scaled)

df_pca = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

df_final = pd.concat([df_no_outliers.reset_index(drop=True), df_pca], axis=1)

explained_variance = pca.explained_variance_ratio_
print("Variância explicada pelos componentes principais:", explained_variance)

print(df_final.head())

## 4.4 Visualização dos dados padronizados

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df_pca['PC1'], df_pca['PC2'], alpha=0.6, color='b')
plt.xlabel('Componente Principal 1 (PC1)')
plt.ylabel('Componente Principal 2 (PC2)')
plt.title('Visualização dos Dados nos Principais Componentes')
plt.grid(True)
plt.show()

# 5. Preparação do Datalake

In [ ]:
parquet_path = './consumo_materiais2019.parquet'
df.to_parquet(parquet_path, index=False)

parquet_path

# 6. Hipóteses e Conclusão

A partir das análises realizadas no Notebook, foi possível extrair os seguintes insights:

1. **Frequência de Manutenção:** A maioria das requisições de manutenção acontecem durante o final do ano. Isto pode indicar um maior fluxo de uso dos sistemas ferroviários, o que pode acarretar na necessidade de manutenções mais recorrentes.

2. **Quantidade de itens adquiridos:** Foi possível perceber que os itens comprados com maior frequência são itens de escritório ou itens de higiene. O que mostra que, aparentemente, a empresa não tem gastos tão frequentes com equipamentos.

3. **Entender o que são requisições de Operação:** O segundo tipo mais frequente de requisição é a de "Operação". No entanto, não existe nenhuma documentação sobre o que isso significa. Nesse sentido, será necessário investigar o significado desta requisição.

4. **Frequência de compras:** A partir da análise de correlação de variáveis, foi possível identificar que os dois picos de compras de materiais acontecem em abril e novembro.